<i>STATUS: Draft</i>

<hr/>

In [121]:
import numpy as np
import sympy as sp
import pickle
from IPython.display import HTML, Image
import ipywidgets as widgets
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
import pandas as pd
import itertools
pd.set_option('display.max_colwidth', None)
import treelib as tr


# function to print latex
def renderListToLatex(e):
    latex_rendering = []

    for i in range(len(e)):
        latex_rendering.append("$$" + sp.latex(e[i]) + "$$")
    
    return(HTML("".join(latex_rendering[0:])))

### Solving Polynomial Equations (21)

<hr/>

<b>Aim</b>: Explore ternary operations on Fuss polygonal subdivisions and thier associated ternary trees

<b>Methodology</b>: Consider the structure of the first row of numbers in the matrix of coefficients returned by the function $C$
<hr/>

<b>Observe</b>: Recall the solution to a general cubic equation from a previous notebook.

$$C(m_2, m_3)  \equiv(-1)^{m_3 + 1} \frac{(2 m_{2} + 3 m_{3})!}{(1 + m_{2} + 2 m_{3})!m_2!m_3!} \frac{c_0^{1 + m_{2} + 2 m_{3}} c_2^{m_2} c_3^{m_3} }{c_1^{2 m_{2} + 3 m_{3} + 1}}$$

<b>Let</b> $F1$ for a function that implements $C$.

In [39]:
def F1(m2, m3, returnCoefficientsOnly = False, returnCoefficientsOnlyWithoutSigns = True, returnCoefficientsAsFactorialStrings = False):
    c_0, c_1, c_2, c_3 = sp.symbols('c_0, c_1, c_2, c_3')
    s1 = (-1)**(m3 + 1)
    s2 = sp.factorial(2 * m2 + 3 * m3)
    s3 = sp.factorial(1 + m2 + 2 * m3) * sp.factorial(m2) * sp.factorial(m3)

    
    s4 = c_0**(1 + m2 + 2 * m3) * c_2**m2 *c_3**m3
    s5 = c_1**(2 * m2 + 3 * m3 + 1)
    
    s7 = str(2 * m2 + 3 * m3) + "!"
    s8 = str(1 + m2 + 2 * m3) + "!" + str(m2) + "!"  + str(m3) + "!"
    
    if returnCoefficientsOnly:
        s6 = s1 * (s2 / s3)
    elif returnCoefficientsOnlyWithoutSigns:
        s6 = (s2 / s3)
    elif returnCoefficientsAsFactorialStrings:
        s6 = str(s7 + " | " + s8)
    else:
        s6 = s1 * (s2 / s3) * (s4 / s5)

    return(s6)

<b>Let</b> $P3$ be a matrix of values relating the part of the formula that calculates unsigned coefficients: $ \frac{(2 m_{2} + 3 m_{3})!}{(1 + m_{2} + 2 m_{3})!m_2!m_3!}$ 

In [40]:
P1 = np.arange(8)
P2 = np.array([[F1(j, i) for i in P1] for j in P1])
P3 = sp.Matrix(P2)
P3

Matrix([
[  1,     1,      3,      12,        55,        273,        1428,         7752],
[  1,     5,     28,     165,      1001,       6188,       38760,       245157],
[  2,    21,    180,    1430,     10920,      81396,      596904,      4326300],
[  5,    84,    990,   10010,     92820,     813960,     6864396,     56241900],
[ 14,   330,   5005,   61880,    678300,    6864396,    65615550,    600900300],
[ 42,  1287,  24024,  352716,   4476780,   51482970,   551170620,   5588372790],
[132,  5005, 111384, 1899240,  27457584,  354323970,  4206302100,  46835886240],
[429, 19448, 503880, 9806280, 159352050, 2283421140, 29804654880, 361913666400]])

Let $P4$ be the first row of numbers in this matrix, also called the Fuss numbers.

In [41]:
P4 = P3[0, :]
P4

Matrix([[1, 1, 3, 12, 55, 273, 1428, 7752]])

<b>Observe</b> The Fuss numbers can be generated by the following function $q$ relating to the OEIS entry A001764

$$ F = \Sigma_{k = 0}^7 \frac{1}{2k + 1}  \binom{3k}{k} q ^k$$

<b>Let</b> $P5$ be a list of numbers generated from the function $F$.

In [122]:
q = sp.symbols('q')
P5 = renderListToLatex([int(1/(2*i + 1) * sp.binomial(3 * i, i)) * q**i for i in range(8)])
P5

<b>Conjecture</b>: the coefficients in $P5$ count the number of diagonal subdivisions of a polygon into quadrilaterals

<b>Observe</b>: This conjecture implies that: 

- There is 1 way to subdivide a 2-gon into quadrialaterals (being a degenerate case)
- There are 55 ways of subdividing a 10-gon into quadrilaterals
- There are 7752 ways of subdividing a 16-gon into quadrilaterals


<b>Observe</b>: In the same manner as introducing a new operation to to join binary trees (using $\overline{\triangledown}$), it is possible to introduce a ternery operation on polygonal subdivisions into quadrilaterals. For this, the symbol $\overline{\square}$ will be used.

<b>Observe</b>: The $\overline{\square}$ operator can be used in the context of $\overline{\square}\text{ }(P, Q, R)$ where $P, Q, R$ represent polygonal sudivisions into quadrilaterals with distinguished roots. 

<b>Observe</b>: The result of this operation will be a new polygonal subdivision.

<b>Strategy</b>: To undertaken operation using $\overline{\square}$ with 3 polygons subdivided into quadrilaterals: 
- Start with quadrilateral with distinguised top which will leave 3 edges $ \overline{\square} $
- For each of the polygons $P, Q$ and $R$, rotate and glue them to a non-distinuised edge of the $\overline{\square}$ operator
- This will create larger polygon.
- The new polygon will also will be divided into quadrilaterals becase the $\overline{\square}$ operator, as well as $P, Q$ and $R$ have been already divided into quadrilaterals.


<b>Observe</b>: This operation is not commutative and not associative.

<b>Observe</b>: The total number of quadrilaterals created by using the $\overline{\square}$ operator is one more than the sum of the quadrilaterals in $P, Q$ and $R$

<b>Let</b> $\psi$ be a mapping from from any subdivided polygons (here denoted $P$) into polynomials in q: 

$$ \psi(P) \equiv q^n $$

Where $n$ is the number of quadrilaterals in the subdivision of $P$.

<b>Observe</b>: It is the case that that: 

$$ \psi(\bar{\square}\text{ }(P, Q, R) = q\psi(P)\psi(Q)\psi(R)$$




<b>Observe</b>: The use of $q$ is equivalent to the use of $|$ in the previous notebook, representing single quadrialteral that gets added when the $\overline{\square}$ operation is undertaken.

<b>Observe</b>: The $\overline{\square}$ operation is a ternary operation. It can be extended to multisets of other subdivided polygons in quadrilaterals

<b>Example</b>: If $A$, $B$ and $C$ are msets of subdivided polygons (into quadrilaterals) a ternary product can be taken by: 

$$ \overline\square \text{ }(A, B, C) \equiv [\overline{\square}(P, Q, R) \text{ where } P \in A, Q \in B, R \in C] $$

<b>Observe</b>: The effect of this operation will take all elements in $A, B, C$ and create a product with all combinations. 

<b>Theorem</b>: 

For an ongoing mset, $A$, which contains each subdivided polygon into quadrilaterals with distinguished edge, including the degenerate polygon, ($|$ the 2 sided polygon), the following relation exists:  

$$ A = | + \overline\square \text{ }(A, B, C) $$


<b>Conjecture</b>: The mapping  $\psi(A)$ to map an mset to monomials in q has the form:  

$$\psi(A)= F = 1 + q + 3q^2 + 12q^3 + 55q^4 + 273q^5 + 1428q^6 + 7752q^7 \ldots $$

<b>Observe</b>: It is possible to apply $\psi(A)$ in relation to the above theorem: 

$$ F = | +  qF^3 $$

<b>Observe</b>: Because all combinations are considered, any ternary tree can be connected to $A, B, C$, (and therefor can be represented by $F$) or will be the the degenerate case $|$


<b>Let</b>: $P6$ be the polynomial in $q$ representing the mapping $\psi(A)$. 

In [123]:
P6 = 1 + q + 3*q**2 + 12*q**3 + 55*q**4 + 273*q**5 + 1428*q**6 + 7752*q**7  
P6

7752*q**7 + 1428*q**6 + 273*q**5 + 55*q**4 + 12*q**3 + 3*q**2 + q + 1

<b>Let</b> $P8$ be an evaluation of the above theorem, $ F = | +  qF^3 $ where $F = 7752 q^{7} + 1428 q^{6} + 273 q^{5} + 55 q^{4} + 12 q^{3} + 3 q^{2} + q + 1$, evaluated to powers of $t$ up to 7

In [124]:
P7 = 1 + q * P6**3
P8 = P7.series(q, 0, 8).removeO()
P8

7752*q**7 + 1428*q**6 + 273*q**5 + 55*q**4 + 12*q**3 + 3*q**2 + q + 1

<b>Observe</b> Consider thie above in the context of general equation, $\Sigma_{b = 0}^7 b_kq^7$ and evaluate using the theorem $ F = | +  qF^3 $

<b>Let</b> $P9$ be a general equatiion of the form $\Sigma_{b = 0}^7 b_kq^7$

In [125]:
b0, b1, b2, b3, b4, b5, b6, b7, q = sp.symbols('b0, b1, b2, b3, b4, b5, b6, b7, q')
P9 = b7 * q**7 + b6 * q**6 + b5 * q **5 + b4 * q**4 + b3 * q**3 + b2 * q**2 + b1 * q + b0
P9

b0 + b1*q + b2*q**2 + b3*q**3 + b4*q**4 + b5*q**5 + b6*q**6 + b7*q**7

<b>Let</b> $P10$ be a power series expansion of $P9$ up to degree 7.

In [106]:
P10 = sp.expand(1 - P9 +  q *  P9**3).series(q, 0, 8).removeO().collect(q)
P10

-b0 + q**7*(3*b0**2*b6 + 6*b0*b1*b5 + 6*b0*b2*b4 + 3*b0*b3**2 + 3*b1**2*b4 + 6*b1*b2*b3 + b2**3 - b7) + q**6*(3*b0**2*b5 + 6*b0*b1*b4 + 6*b0*b2*b3 + 3*b1**2*b3 + 3*b1*b2**2 - b6) + q**5*(3*b0**2*b4 + 6*b0*b1*b3 + 3*b0*b2**2 + 3*b1**2*b2 - b5) + q**4*(3*b0**2*b3 + 6*b0*b1*b2 + b1**3 - b4) + q**3*(3*b0**2*b2 + 3*b0*b1**2 - b3) + q**2*(3*b0**2*b1 - b2) + q*(b0**3 - b1) + 1

<b>Let</b> $P11$ be each term in $P10$ set to 0 in order to solve for all values of $b_i$

In [126]:
P11 = [sp.Eq(P10.args[i], 0).subs(q, 1) for i in range(2, len(list(P10.args)))]
P11.insert(0, sp.Eq(1 - b0, 0))
renderListToLatex(P11)

<b>Let</b>: Let $P12$ be a solution for all values of $b_i$

In [128]:
sp.solve(P11, (b0, b1, b2, b3, b4, b5, b6, b7))

[(1, 1, 3, 12, 55, 273, 1428, 7752)]

<b>Observe</b>: This shows that the Fuss numbers are a unique solution.

<b>Todo</b>: Fugure out recursive formula to explain the structure of numbers being seen in $P11$